# Usages by Cycles and Groups

### Introduction

The purpose of this script is to generate the data table of usage information by each group in each year. The data source is from the tables "midway_usage" and "midway_usage_archived" of RCC AcctTool database. The script will generate the 6 yearly data file 'years_detail_usages.csv' with six columns: 
 - cycle: 201X means subtotals from job records ended between 10/01/201(X-1) to 09/30/201X
 - account: account name
 - jobs: total number of jobs
 - users: number of users having job records
 - su: total SUs being charged
 - cpuhour: total cpu(core)-hours being consumed. 

#### Sum up the jobs, users, usages and cuphours to get a data file 'usages_by_cycle_group.csv'. 
#### Each row is a record of usage data for a group in a year.







In [1]:
%run include/required_libs.py
%matplotlib inline

We firstly get yearly information from database 
2013-2017

In [2]:
cycles = list(range(2013,2019))

In [4]:
for year in cycles:
    a = '''select account,count(*) as `jobs`,
            sum(`service_units_charged`) as `su`,
            count(distinct `user`) as `users`,
            avg(ncpus) as `avg_cores`,
            avg(nnodes) as `avg_nodes`,
            sum(`cputimeraw`)/3600 as `cpuhours` '''
    
    if year == cycles[-1]:
        b = '''from midway_usage '''
        
    else:
        b = '''from midway_usage_archived '''
        
    c = '''where (partition='sandyb' or partition='broadwl') and end >' '''

    d = '''-09-30' and end <' '''

    e = '''-10-01 00:00:00' 
        group by account 
        order by `su` desc'''
    
    yearly = pd.read_sql(a + b + c + str(year-1) + d + str(year) + e, cnx)
    yearly.to_csv('rawdata/usage_by_group_' + str(year) + '.csv')

Then, add cycle name and append to get 2013-2018 usages_by_cycle_group

In [8]:
usages_by_cycle_group = pd.DataFrame()

for year in cycles:
    yearly = pd.read_csv('rawdata/usage_by_group_' + str(year) + '.csv')
    yearly['cycle'] = year
    usages_by_cycle_group = usages_by_cycle_group.append(yearly)
    

In [9]:
usages_by_cycle_group.to_csv('rawdata/usages_by_cycle_group.csv', index=False)